# Лабораторная работа №4
Optuna

In [3]:
import logging
import sys

import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
import plotly
import numpy as np

# Функция Бута
$f(x,y) = (x + 2y - 7)^2 + (2x + y - 5)^2$

In [4]:
def Buta(x: np.ndarray, y: np.ndarray) -> np.float64:
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2

def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    return Buta(x, y)

Поиск гиперпараметров

In [5]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2023-01-16 19:17:53,774] A new study created in memory with name: no-name-7b407405-9701-41d3-81d9-12f5c0e21cf4
[I 2023-01-16 19:17:53,779] Trial 0 finished with value: 75.94383015108217 and parameters: {'x': 6.544777661264703, 'y': -3.46577894569206}. Best is trial 0 with value: 75.94383015108217.
[I 2023-01-16 19:17:53,782] Trial 1 finished with value: 1253.120430827826 and parameters: {'x': -8.853607718482772, 'y': -3.802834629466383}. Best is trial 0 with value: 75.94383015108217.
[I 2023-01-16 19:17:53,783] Trial 2 finished with value: 919.1966053489944 and parameters: {'x': -5.136220131868159, 'y': -5.140321575506201}. Best is trial 0 with value: 75.94383015108217.
[I 2023-01-16 19:17:53,785] Trial 3 finished with value: 178.45797763443954 and parameters: {'x': 7.377034960820165, 'y': 2.4865774066447344}. Best is trial 0 with value: 75.94383015108217.
[I 2023-01-16 19:17:53,787] Trial 4 finished with value: 3.259823762436021 and parameters: {'x': 1.5037136220983456, 'y': 3.3457

In [6]:
best_params = study.best_params
found_x = best_params["x"]
found_y = best_params["y"]
print(f'''f(1,3) = 0
X: {found_x}
Y: {found_y}
F(X,Y): {Buta(found_x, found_y)}''')


f(1,3) = 0
X: 0.8206176095111877
Y: 3.041662097054127
F(X,Y): 0.10978128924337152


# Использование Optuna для нахождения гиперпараметров датасета Iris

In [7]:
def objective(trial):
    iris = sklearn.datasets.load_iris()
    classes = list(set(iris.target))
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        iris.data, iris.target, test_size=0.25, random_state=0
    )
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)
    clf = sklearn.linear_model.SGDClassifier(alpha=alpha)
    for step in range(100):
        clf.partial_fit(train_x, train_y, classes=classes)
        intermediate_value = 1.0 - clf.score(valid_x, valid_y)
        trial.report(intermediate_value, step)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return 1.0 - clf.score(valid_x, valid_y)

In [8]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

[I 2023-01-16 19:17:55,161] A new study created in memory with name: no-name-f58918a8-e57b-4ea5-815f-3a43f188bd3c


A new study created in memory with name: no-name-f58918a8-e57b-4ea5-815f-3a43f188bd3c


[I 2023-01-16 19:17:55,415] Trial 0 finished with value: 0.07894736842105265 and parameters: {'alpha': 0.004985919325144095}. Best is trial 0 with value: 0.07894736842105265.


Trial 0 finished with value: 0.07894736842105265 and parameters: {'alpha': 0.004985919325144095}. Best is trial 0 with value: 0.07894736842105265.


[I 2023-01-16 19:17:55,718] Trial 1 finished with value: 0.13157894736842102 and parameters: {'alpha': 0.00012371714817355944}. Best is trial 0 with value: 0.07894736842105265.


...

Trial 17 finished with value: 0.21052631578947367 and parameters: {'alpha': 0.01362390526336521}. Best is trial 3 with value: 0.02631578947368418.


[I 2023-01-16 19:17:57,052] Trial 18 pruned. 


Trial 18 pruned. 


[I 2023-01-16 19:17:57,067] Trial 19 pruned. 


Trial 19 pruned. 


# Вывод итогов

In [10]:
plot_optimization_history(study)

![](OHP.png)

In [11]:
plot_intermediate_values(study)

![](IVP.png)

In [12]:
plot_parallel_coordinate(study)

![](PCP.png)

In [13]:
plot_slice(study)

![](SP.png)

In [14]:
plot_param_importances(study)

![](HI.png)

In [15]:
plot_edf(study)

![](EDFP.png)